## Input the Parameters of the Starlette-SLR GEODYN run:

In [1]:
import pandas as pd
import numpy as np

In [2]:
ll /data/runs_geodyn/iss/results/msis2/msis2_accelon/IIEOUT/


total 346644
-rw-rw-r--. 1 m_geodyn 354960125 Feb 19 20:05 iss190420_2dy.goco05s


In [3]:


########################
# INPUT PARAMETERS:
########################
sat_file = 'iss'
arc = '190420_2dy'
grav_id ='goco05s' 
local_path = '/data/analysis/ISS_GPS_analysis/'
SAT_ID = 9806701
accel_card = 'accelon'


#######################################
# PATH TO DENSITY MODEL RUN of Choice:
#######################################
msis2_model = 'msis2'
path_to_msis2 = '/data/runs_geodyn/'+sat_file+'/results/'+ msis2_model+'/'+ msis2_model+ '_' +accel_card+'/'



### Call the prep-function that loads the data

This may take a few minutes to load.

#### MSIS2

In [4]:
# %load_ext autoreload
# %autoreload 2

# import sys  
# sys.path.insert(0, '/data/analysis/util_funcs/py_geodynreader_gps/')

# from a_ReadISS import ReadISS

# AdjustedParams, Trajectory, Density, Resids = ReadISS(arc, 
#                                                            sat_file,
#                                                            grav_id, 
#                                                            local_path, 
#                                                            path_to_msis2,
#                                                            True)

In [5]:
# WONKY INPUTS:
path_to_data = path_to_msis2
AccelStatus = True


import sys  
sys.path.insert(0, '/data/analysis/notebooks/util_funcs/py_geodynreader_gps/')
SAT_ID = 9806701
sat = sat_file
file_name =  sat + arc + '.'+ grav_id
print('The base file name for this arc is:',file_name,'\n' )
import os

iieout_file  = '/data/analysis/IIEOUT_'+file_name   #path_to_data + 'IIEOUT/'+ file_name
traj_file  = '/data/analysis/TRAJ_'+file_name   #path_to_data + 'IIEOUT/'+ file_name


# from b_ReadISS import Save_AdjustedParameters_ISS
# SatMain_AdjustedParams = Save_AdjustedParameters_ISS(SAT_ID, iieout_file, AccelStatus)


The base file name for this arc is: iss190420_2dy.goco05s 



In [42]:
def make_datetime_column_resids(resid_df, YR):
    VERBOSE_timer = False
    if VERBOSE_timer == True:
        import time
        start = time.time()
    else:
        pass

    # resid_df['YYMMDD'] = resid_df['YYMMDD'].astype(int).astype(str)
    # resid_df['HHMM'] =  resid_df['HHMM'].astype(int)
    # resid_df['SEC_UTC'] =  resid_df['SEC_UTC'].astype(float).astype(str)

    resid_df['YYMMDD'] = resid_df['YYMMDD'].astype(int).astype(str)
    resid_df['HHMM'] = resid_df['HHMM'].astype(int).astype(str)


    timeHHMM = [] 
    for i,val in enumerate(resid_df['HHMM']):
    #     print(val)
    #     print(len(val))
        if len(val) == 3:
            timehhmm_val = '0'+ val
            timeHHMM.append(timehhmm_val)
        elif len(val) == 2:
            timehhmm_val = '00'+ val
            timeHHMM.append(timehhmm_val)
        elif len(val) == 1:
            timehhmm_val = '000'+ val
            timeHHMM.append(timehhmm_val)
        elif len(val) == 4:
            timehhmm_val = val
            timeHHMM.append(timehhmm_val)
    #     else:
    #         print('NO WORK')


    #     np.shape(timeHHMM)
    resid_df['timeHHMM'] = timeHHMM

    year  = []
    month = []
    day   = []
    hours  = []
    minutes = []
    secs  = []
    microsecs = []
    for i,val in enumerate(resid_df['YYMMDD']):
    #         print(val[:2])
        if YR < 10:
            year.append('200' + val[:1])
            month.append(val[1:3])
            day.append(val[3:])

            hours.append(resid_df['timeHHMM'][i][:2])
            minutes.append(resid_df['timeHHMM'][i][2:4])

            secs.append(resid_df['SEC_UTC'][i][:2])
            microsecs.append(resid_df['SEC_UTC'][i][3:])
        else:
            year.append('20' + val[:2])
            month.append(val[2:4])
            day.append(val[4:])

            hours.append(resid_df['timeHHMM'][i][:2])
            minutes.append(resid_df['timeHHMM'][i][2:4])

            secs.append(resid_df['SEC_UTC'][i][:2])
            microsecs.append(resid_df['SEC_UTC'][i][3:])

    resid_df['year']  = year
    resid_df['month'] = month
    resid_df['day']   = day
    resid_df['hours']  = hours
    resid_df['minutes'] = minutes
    resid_df['secs']  = secs
    resid_df['microsecs'] = microsecs
    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Loop through and extract indiv date vals:",elapsed)
    else:
        pass

    fix_decimal = []
    for i,val in enumerate(resid_df['secs'].astype(str)):
    #     print(i,val)
        if val.find('.') == 1:
    #             print(i, val)
            fix_decimal.append( '0'+val[:-1])
    #             print(newval)
        else:
            fix_decimal.append( val)

    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
        print("Fix decimals in the seconds column:",elapsed)
    else:
        pass

    year= list(map(int, resid_df['year'].values))
    month= list(map(int, resid_df['month'].values))
    day= list(map(int, resid_df['day'].values))
    hour= list(map(int, resid_df['hours'].values))
    minute = list(map(int, resid_df['minutes'].values))
    second = list(map(int, fix_decimal))
    microsecond= list(map(int, resid_df['microsecs'].values))

    DATE = list(map(datetime, year,month, day, hour,minute,second,microsecond ))

    if VERBOSE_timer == True:
        end = time.time()
        elapsed = end - start
    #         print("Put all dates in a single column:",elapsed)
    else:
        pass


    return(DATE)


In [45]:
def read_ascixyz_gps(traj_file):
    '''
    This function reads the ascii XYZ trajectory data.

    The ascii XYZ card SHOULD only be giving the FINAL iteration 
    of the GEODYN RUN.

    In a GPS run, there are many satellites being tracked. 
    This function returns a dataframe with time series of the trajectory 
    data with the satellite as one of the columns.
    '''
    #
    # determine where the first data header is (want to skip the file header)
    with open(traj_file, 'r') as f:
        for line_no, line in enumerate(f):
            if 'SPACECRAFT EPHEMERIS FOR SATELLITE' in line:
                first_header_line = line_no
                break
    #
    # read the total number of lines
    line_total = len(open(traj_file).readlines())
    #
    # initialize some lists for iteratively storing
    list_YYMMDD   = []
    list_HHMM     = []
    list_SECONDS  = []
    list_X        = []
    list_Y        = []
    list_Z        = []
    list_XDOT     = []
    list_YDOT     = []
    list_ZDOT     = []
    list_LAT      = []
    list_LONG     = []
    list_HEIGHT   = []
    list_sat_id   = []
    #
    # loops through all lines in the file from the first 
    # data header to the last file line.
    for i,val in enumerate(np.arange(first_header_line+1,line_total+1)):
        line = linecache.getline(traj_file,val)
        #
        # if the line contains the below text, the satellite is changing
        # save the new sat ID.
        if 'SPACECRAFT EPHEMERIS FOR SATELLITE' in line:
            sat_id = line[46:54]
        #
        # the following keywords denote a header in the file and these lines should be skipped
        if "SPACECRAFT" in line:
            continue
        elif "GREENWICH" in line:
            continue
        elif "YYMMDD" in line:
            continue
        #
        # If the line is NOT a header, save the data out.
        # Note that at this point in the loop we are saving the 
        # data for the above Sat ID.  When we get to a line 
        # with the 'SPACECRAFT EPHEMERIS FOR SATELLITE' text,
        # the satellite ID will change and the data we save 
        # out will be for that new satellite.
        else:
            YYMMDD  = line[1:8]
            HHMM    = line[8:13]
            SECONDS = line[13:23] 
            X       = line[23:36]
            Y       = line[36:49]
            Z       = line[49:62]
            XDOT    = line[62:74]
            YDOT    = line[74:86]
            ZDOT    = line[86:98]
            LAT     = line[98:109]
            LONG    = line[109:120]
            HEIGHT  = line[120:132]

            # store in a list to catch 'em all
            list_YYMMDD.append(YYMMDD)
            list_HHMM.append(HHMM)
            list_SECONDS.append(SECONDS)
            list_X.append(X)
            list_Y.append(Y)
            list_Z.append(Z)
            list_XDOT.append(XDOT)
            list_YDOT.append(YDOT)
            list_ZDOT.append(ZDOT)
            list_LAT.append(LAT)
            list_LONG.append(LONG)
            list_HEIGHT.append(HEIGHT)
            list_sat_id.append(sat_id)

    #
    # Save as a dataframe
    traj_xyz_df = pd.DataFrame(data={ 'YYMMDD':list_YYMMDD,
                                    'HHMM'    :list_HHMM,
                                    'SEC_UTC' :list_SECONDS,
                                    'SAT_ID'  :list_sat_id,
                                    'X'       :list_X,
                                    'Y'       :list_Y,
                                    'Z'       :list_Z,
                                    'XDOT'    :list_XDOT,
                                    'YDOT'    :list_YDOT,
                                    'ZDOT'    :list_ZDOT,
                                    'LAT'     :list_LAT,
                                    'LONG'    :list_LONG,
                                    'HEIGHT'  :list_HEIGHT})

    date = make_datetime_column_resids(traj_xyz_df, 19)
    traj_xyz_df['Date'] = date


    traj_xyz_df['X']  = traj_xyz_df['X'].astype(float)
    traj_xyz_df['Y']  = traj_xyz_df['Y'].astype(float)
    traj_xyz_df['Z']  = traj_xyz_df['Z'].astype(float)
    traj_xyz_df['XDOT']  = traj_xyz_df['XDOT'].astype(float)
    traj_xyz_df['YDOT']  = traj_xyz_df['YDOT'].astype(float)
    traj_xyz_df['ZDOT']  = traj_xyz_df['ZDOT'].astype(float)
    traj_xyz_df['LAT']  = traj_xyz_df['LAT'].astype(float)
    traj_xyz_df['LONG']  = traj_xyz_df['LONG'].astype(float)
    traj_xyz_df['HEIGHT']  = traj_xyz_df['HEIGHT'].astype(float)



    return(traj_xyz_df)

In [46]:
traj_xyz_df

,YYMMDD,HHMM,SEC_UTC,SAT_ID,X,Y,Z,XDOT,YDOT,ZDOT,...,HEIGHT,timeHHMM,year,month,day,hours,minutes,secs,microsecs,Date
0,190420,2129,42.000000,6163016,-12517952.80,8987515.75,21411971.91,-1478.58507,-3549.60279,651.61171,...,20016760.90,2129,2019,04,20,21,29,42,000000,2019-04-20 21:29:42
1,190420,2129,42.000000,6163016,-12517952.80,8987515.75,21411971.91,-1478.58507,-3549.60279,651.61171,...,20016760.90,2129,2019,04,20,21,29,42,000000,2019-04-20 21:29:42
2,190420,2129,42.000000,6163016,-12517952.80,8987515.75,21411971.91,-1478.58507,-3549.60279,651.61171,...,20016760.90,2129,2019,04,20,21,29,42,000000,2019-04-20 21:29:42
3,190420,2129,42.000000,5461024,16327458.63,14348999.95,-14766190.58,-586.46789,3127.39564,2281.61495,...,19906364.90,2129,2019,04,20,21,29,42,000000,2019-04-20 21:29:42
4,190420,2129,42.000000,5461024,16327458.63,14348999.95,-14766190.58,-586.46789,3127.39564,2281.61495,...,19906364.90,2129,2019,04,20,21,29,42,000000,2019-04-20 21:29:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10876,190422,229,42.000000,6468096,-4142369.33,-16046129.70,20716983.96,3766.22914,198.69967,904.02040,...,20164715.72,0229,2019,04,22,02,29,42,000000,2019-04-22 02:29:42
10877,190422,229,42.000000,6468096,-4142369.33,-16046129.70,20716983.96,3766.22914,198.69967,904.02040,...,20164715.72,0229,2019,04,22,02,29,42,000000,2019-04-22 02:29:42
10878,190422,229,42.000000,6573106,8581437.68,12802368.25,-21584724.94,-2847.74376,2604.91180,392.45772,...,20158508.91,0229,2019,04,22,02,29,42,000000,2019-04-22 02:29:42
10879,190422,229,42.000000,6573106,8581437.68,12802368.25,-21584724.94,-2847.74376,2604.91180,392.45772,...,20158508.91,0229,2019,04,22,02,29,42,000000,2019-04-22 02:29:42


In [ ]:
A[0]

In [ ]:

   

# SatID_dict = {}
# iteration = str(int(numiters))
# for val_sat in SatIDs_ascii:
#     # print(val_sat) 
#     lines = []
#     text = str(val_sat) + " OF ARC  1 FOR INNER ITERATION  "+ iteration
#     with open(self.ascii_xyz_file, 'r') as f:
#         for line_no, line in enumerate(f):
#             if text in line:
#                 lines.append(line_no)


#         SatID_dict[val_sat] = lines


# #  Next, we need to loop through and grab the data.
# #  Because of the weird formatting, we search for the satellite header.  
# #  If the header line starts with 1 the next 3 lines are headers and we skip them
# #  If the header line starts with 0, the next line has the data

# data_dict = {}
# isat = self.choose_sat
# iii = 0

# # for iii, isat in enumerate(SatID_dict):

# B = pd.DataFrame(data={'YYMMDD'   :[], # DATE  GREENWICH TIME
#                       'HHMM'    :[],
#                       'SECONDS' :[],
#                       'X'       :[], # INERTIAL CARTESIAN COORDINATES
#                       'Y'       :[],
#                       'Z'       :[],
#                       'XDOT'    :[], # INERTIAL CARTESIAN VELOCITY
#                       'YDOT'    :[],
#                       'ZDOT'    :[],
#                       'LAT'     :[], # GEODETIC EAST SPHEROID
#                       'LONG'    :[],
#                       'HEIGHT'  :[]})

# # print(SatID_dict)
# # for iline in SatID_dict[isat]:

# with open(self.ascii_xyz_file, 'r') as f: 
# #     for _ in range(iline): 
# #         f.readline() 
# #     line = f.readline() 

#         ephems_csv = pd.read_csv(self.ascii_xyz_file, 
#                             skiprows = SatID_dict[isat][0]+1,
# #                             nrows =  SatID_dict[isat][-1]+,
#                             names = ['YYMMDD',
#                                     'HHMM',
#                                     'SECONDS',
#                                     'X',
#                                     'Y',
#                                     'Z',
#                                     'XDOT',
#                                     'YDOT',
#                                     'ZDOT',
#                                     'LAT',
#                                     'LONG',
#                                     'HEIGHT',
#                                   ],
#                             sep = '\s+',
#                             )

# B = pd.DataFrame(ephems_csv)
# index_list = []
# for index, row in B.iterrows():
#     try:
#         float(row['SECONDS'])
#     except:
#         index_list.append(index)
#         continue

# C=B.drop(index_list)

# data_dict[isat] = C
# # print(C)
# date_isat = read_ascii_xyz.make_datetime_column(data_dict[isat], VERBOSE_timer=False) 
# data_dict[isat]['Date'] = date_isat






In [ ]:
# class read_ascii_xyz:
'''
This function reads the ascii XYZ trajectory data.

? For the GPS satellite run, this is given in terms of the satellites that are tracking the ISS?



'''


# # def find_satIDs(self):
#   '''
#   This function loops through the ascii xyz file and returns the satellite ID
#   numbers by identifying all the unique satellite IDs
#   '''
        
# numiters = read_ascii_xyz.iteration_number(self)
# text_find_sats = "ARC  1 FOR INNER ITERATION  %d OF GLOBAL ITERATION 1" % int(numiters)
# allsats = []
# with open(self.ascii_xyz_file, 'r') as f:

#     for line_no, line in enumerate(f):
#         if text_find_sats in line:
#             allsats.append((int(line[45:54])))

# SatIDs = []
# for sat in allsats:
#     if sat not in SatIDs:
#         SatIDs.append(sat)
# #   return SatIDs

#     def get_single_sat_data(self):
        '''
        This function loops through only the final iteration of the axi_xyz file, 
        and returns a dictionary that contains all the data for one single satellite.
        In this function it is a satellite chosen by the user.

        Eventually this should be update to return info for ALL satellites.
        '''
        # First need to construct a dictionary that has all the line numbers where each 
        # satellite appears:

        
        numiters = read_ascii_xyz.iteration_number(self)
        SatIDs_ascii = read_ascii_xyz.find_satIDs(self)
        SatID_dict = {}
        iteration = str(int(numiters))
        for val_sat in SatIDs_ascii:
            # print(val_sat) 
            lines = []
            text = str(val_sat) + " OF ARC  1 FOR INNER ITERATION  "+ iteration
            with open(self.ascii_xyz_file, 'r') as f:
                for line_no, line in enumerate(f):
                    if text in line:
                        lines.append(line_no)


                SatID_dict[val_sat] = lines


        #  Next, we need to loop through and grab the data.
        #  Because of the weird formatting, we search for the satellite header.  
        #  If the header line starts with 1 the next 3 lines are headers and we skip them
        #  If the header line starts with 0, the next line has the data

        data_dict = {}
        isat = self.choose_sat
        iii = 0

        # for iii, isat in enumerate(SatID_dict):

        B = pd.DataFrame(data={'YYMMDD'   :[], # DATE  GREENWICH TIME
                              'HHMM'    :[],
                              'SECONDS' :[],
                              'X'       :[], # INERTIAL CARTESIAN COORDINATES
                              'Y'       :[],
                              'Z'       :[],
                              'XDOT'    :[], # INERTIAL CARTESIAN VELOCITY
                              'YDOT'    :[],
                              'ZDOT'    :[],
                              'LAT'     :[], # GEODETIC EAST SPHEROID
                              'LONG'    :[],
                              'HEIGHT'  :[]})

        # print(SatID_dict)
        # for iline in SatID_dict[isat]:

        with open(self.ascii_xyz_file, 'r') as f: 
        #     for _ in range(iline): 
        #         f.readline() 
        #     line = f.readline() 

                ephems_csv = pd.read_csv(self.ascii_xyz_file, 
                                    skiprows = SatID_dict[isat][0]+1,
        #                             nrows =  SatID_dict[isat][-1]+,
                                    names = ['YYMMDD',
                                            'HHMM',
                                            'SECONDS',
                                            'X',
                                            'Y',
                                            'Z',
                                            'XDOT',
                                            'YDOT',
                                            'ZDOT',
                                            'LAT',
                                            'LONG',
                                            'HEIGHT',
                                          ],
                                    sep = '\s+',
                                    )

        B = pd.DataFrame(ephems_csv)
        index_list = []
        for index, row in B.iterrows():
            try:
                float(row['SECONDS'])
            except:
                index_list.append(index)
                continue

        C=B.drop(index_list)

        data_dict[isat] = C
        # print(C)
        date_isat = read_ascii_xyz.make_datetime_column(data_dict[isat], VERBOSE_timer=False) 
        data_dict[isat]['Date'] = date_isat

        return data_dict






In [ ]:
# def make_datetime_column(isat_data, VERBOSE_timer):

#     if VERBOSE_timer == True:
#       import time
#       start = time.time()
#     else:
#       pass

#     timeHHMM = [] 
#     for i,val in enumerate(isat_data['HHMM'].values.astype(int)):
#         # print(len(str(val)))
#         if len(str(val)) == 3:
#             timehhmm_val = '0'+ str(val)
#             timeHHMM.append(timehhmm_val)
#         if len(str(val)) == 2:
#             timehhmm_val = '00'+ str(val)
#             timeHHMM.append(timehhmm_val)
#         if len(str(val)) == 4:
#             timehhmm_val = str(val)
#             timeHHMM.append(timehhmm_val)
#         if len(str(val)) == 1:
#             timehhmm_val = '000'+ str(val)
#             timeHHMM.append(timehhmm_val)
#     # print(val)  
#     # print('1!!!!', np.shape(timeHHMM))

#     isat_data['timeHHMM'] = timeHHMM

#     year  = []
#     month = []
#     day   = []
#     hours  = []
#     minutes = []
#     secs  = []
#     microsecs = []
#     for i,val in enumerate(isat_data['YYMMDD'].values.astype(int).astype(str)):
#         # print(val)
#         year.append('200' + val[:1])
#         # print(year)
#         month.append(val[1:3])
#         day.append(val[3:])
#         # print('HERE',isat_data['timeHHMM'].values.astype(str)[i][:2])
#         hours.append(isat_data['timeHHMM'].values.astype(str)[i][:2])
#         minutes.append(isat_data['timeHHMM'].values.astype(str)[i][2:4])
#         secs.append(isat_data['SECONDS'].values.astype(str)[i][:2])
#         # microsecs.append(isat_data['Sec-UTC-R'][i][3:])

#     isat_data['year']  = year
#     isat_data['month'] = month
#     isat_data['day']   = day
#     isat_data['hours']  = hours
#     isat_data['minutes'] = minutes
#     isat_data['secs']  = secs
#     # isat_data['microsecs'] = microsecs
#     if VERBOSE_timer == True:
#       end = time.time()
#       elapsed = end - start
#       print("Loop through and extract indiv date vals:",elapsed)
#     else:
#       pass

#     fix_decimal = []
#     for i,val in enumerate(isat_data['secs'].astype(str)):
#     #     print(i,val)
#         if val.find('.') == 1:
#     #             print(i, val)
#             fix_decimal.append( '0'+val[:-1])
#     #             print(newval)
#         else:
#             fix_decimal.append( val)

#     if VERBOSE_timer == True:
#       end = time.time()
#       elapsed = end - start
#       print("Fix decimals in the seconds column:",elapsed)
#     else:
#       pass

#     year= list(map(int, isat_data['year'].values))
#     month= list(map(int, isat_data['month'].values))
#     day= list(map(int, isat_data['day'].values))
#     hour= list(map(int, isat_data['hours'].values))
#     minute = list(map(int, isat_data['minutes'].values))
#     second = list(map(int, fix_decimal))
#     DATE = list(map(datetime, year,month, day, hour,minute,second ))

#     if VERBOSE_timer == True:
#       end = time.time()
#       elapsed = end - start
#       print("Put all dates in a single column:",elapsed)
#     else:
#       pass
#     return(DATE)
